In [1]:
pip install torchvision

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
from functools import reduce

import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.layers = nn.Sequential(                                     # mnist的输入图片大小是 1 * 28 * 28
            nn.Flatten(),                                                # batch * 28 * 28
            nn.Linear(in_features=28 * 28, out_features=240),               
            nn.Linear(in_features=240, out_features=10)                   # 最后分成10类
        )

    def forward(self, x):
        # 用reduce把所有层的函数合并为一个函数
        # 这样就不需要手动把layers(1)的输出传给layers(2)、layers(2)的输出传给layers(3)...
        # 当layers个数和顺序变化时，此处就不需要改变
        mergeFunction = reduce(lambda f1, f2: lambda input: f2(f1(input)), self.layers)
        return mergeFunction(x)

# 训练用的超参
epochNum = 10
batchSize = 100
lr = 0.001

# array 转 tensor
transform = transforms.ToTensor()

# 训练集
trainSet = tv.datasets.MNIST(
    root='./train',
    train=True,
    download=True,
    transform=transform)

# 训练集转迭代器做训练
trainLoader = DataLoader(
    trainSet,
    batch_size=batchSize,
    shuffle=True)

# 构建LeNet模型
model = LeNet()
# 用随机梯度下降做优化器
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# 用交叉熵做分类损失函数
lossFunction = nn.CrossEntropyLoss()

# 测试集
testSet = tv.datasets.MNIST(
    root='./test',
    train=False,
    download=True,
    transform=transform)

# 测试集转迭代器来计算准确率
testLoader = DataLoader(
    testSet,
    batch_size=batchSize,
    shuffle=False)

def printAccuracy():
    with torch.no_grad():
        correct = 0
        total = 0
        for (images, labels) in testLoader:
            outputs = model(images)
            # 取得分最高的那个类
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        accuracy = correct * 100.0 / total
        print('当前准确率: %.3f%%' % accuracy)


printAccuracy()

# 开始迭代训练
for epoch in range(0, epochNum):
    totalLoss = 0.0
    for batch, (imgs, labels) in enumerate(trainLoader):
        # 清理梯度
        optimizer.zero_grad()
        # 根据LeNet计算输出
        outputs = model(imgs)
        # 计算loss
        loss = lossFunction(outputs, labels)
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        # 每个100个batch之内计算loss之和所谓当前100个batch的loss
        totalLoss += loss.item()
        if batch % 100 == 99:
            print('epoch=%d batch=%d loss: %.03f' % (epoch + 1, batch + 1, totalLoss / 100))
            totalLoss = 0.0

    printAccuracy()

# 模型存档
torch.save(model.state_dict(), 'lenet.model')
print("程序结束")

当前准确率: 13.650%
epoch=1 batch=100 loss: 2.098
epoch=1 batch=200 loss: 1.674
epoch=1 batch=300 loss: 1.292
epoch=1 batch=400 loss: 1.022
epoch=1 batch=500 loss: 0.860
epoch=1 batch=600 loss: 0.747
当前准确率: 85.430%
epoch=2 batch=100 loss: 0.663
epoch=2 batch=200 loss: 0.606
epoch=2 batch=300 loss: 0.576
epoch=2 batch=400 loss: 0.540
epoch=2 batch=500 loss: 0.515
epoch=2 batch=600 loss: 0.497
当前准确率: 88.350%
epoch=3 batch=100 loss: 0.473
epoch=3 batch=200 loss: 0.461
epoch=3 batch=300 loss: 0.449
epoch=3 batch=400 loss: 0.442
epoch=3 batch=500 loss: 0.427
epoch=3 batch=600 loss: 0.422
当前准确率: 89.410%
epoch=4 batch=100 loss: 0.410
epoch=4 batch=200 loss: 0.411
epoch=4 batch=300 loss: 0.411
epoch=4 batch=400 loss: 0.384
epoch=4 batch=500 loss: 0.390
epoch=4 batch=600 loss: 0.384
当前准确率: 89.880%
epoch=5 batch=100 loss: 0.371
epoch=5 batch=200 loss: 0.377
epoch=5 batch=300 loss: 0.373
epoch=5 batch=400 loss: 0.381
epoch=5 batch=500 loss: 0.366
epoch=5 batch=600 loss: 0.366
当前准确率: 90.320%
epoch=6 ba